## Bayes'sches Netz zur Bestimmung des nächsten Vorschlags mit höchster Wahrscheinlichkeit

In [ ]:
import numpy as np

import pandas as pd

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.estimators import MaximumLikelihoodEstimator

import math

import json

In [ ]:
words = pd.read_csv("words.csv")

model = BayesianNetwork([
    ("first", "second"), 
    ("third", "second"), 
    ("third", "forth"), 
    ("fifth", "forth")])

model.fit(words, estimator=MaximumLikelihoodEstimator)

infer = VariableElimination(model)

In [ ]:
def get_suggestion_word(suggestion, evidence):
    word = ["", "", "", "", ""]
    
    if "first" in evidence:
        word[0] = evidence["first"]
    else: 
        word[0] = suggestion["first"]
        
    if "second" in evidence:
        word[1] = evidence["second"]
    else:
        word[1] = suggestion["second"]
    
    if "third" in evidence:
        word[2] = evidence["third"]
    else:
        word[2] = suggestion["third"]
    
    if "forth" in evidence:
        word[3] = evidence["forth"]
    else:
        word[3] = suggestion["forth"]
    
    if "fifth" in evidence:
        word[4] = evidence["fifth"]
    else:
        word[4] = suggestion["fifth"]
        
    return word

def word_is_valid(word, words, must_contain=[], must_not_contain=[], must_not_contain_at={}):
    if "".join(str(char) for char in word) in words:
        for letter in must_contain:
            if not letter in word:
                return False
            
        for letter in must_not_contain:
            if letter in word:
                return False
        
        if "first" in must_not_contain_at and word[0] in must_not_contain_at["first"]:
            return False
                
        if "second" in must_not_contain_at and word[1] in must_not_contain_at["second"]:
            return False
                
        if "third" in must_not_contain_at and word[2] in must_not_contain_at["third"]:
            return False
                
        if "forth" in must_not_contain_at and word[3] in must_not_contain_at["forth"]:
            return False
                
        if "fifth" in must_not_contain_at and word[4] in must_not_contain_at["fifth"]:
            return False
        
        return True
    else:
        return False
    
def get_suggestion(variables, evidence, must_contain=[], must_not_contain=[], must_not_contain_at=[]):
    q = infer.query(variables, evidence=evidence, show_progress=False)
    
    count_predictions = len(q.values.flatten()[q.values.flatten() != 0])
    max_value_indices = (-q.values.flatten()).argsort()[:count_predictions]

    with open("wordle-at-words.json", "r") as f:
        words = json.load(f)
        
        for max_value_index in max_value_indices:
            indices = np.unravel_index(max_value_index, q.values.shape)

            suggestion = {}

            for index, variable in enumerate(q.variables):
                suggestion[variable] = model.get_cpds(variable).state_names[variable][indices[index]]

            word = get_suggestion_word(suggestion, evidence)

            if word_is_valid(word, words, must_contain, must_not_contain, must_not_contain_at):
                return word
            
    return []

def get_first_suggestion():
    with open("wordle-at-words.json", "r") as f:
        words_json = json.load(f)

    first = infer.map_query(["first"], show_progress=False)["first"]
    second = infer.map_query(["second"], evidence={"first": first}, show_progress=False)["second"]
    third = infer.map_query(["third"], evidence={"first": first, "second": second}, show_progress=False)["third"]
    forth = infer.map_query(["forth"], evidence={"first": first, "second": second, "third": third}, show_progress=False)["forth"]
    fifth = infer.map_query(["fifth"], evidence={"first": first, "second": second, "third": third, "forth": forth}, show_progress=False)["fifth"]

    if word_is_valid("".join([first,second,third,forth,fifth]), words_json):
        return "".join[first,second,third,forth,fifth]
    elif len(words[
            (words["first"] == first) & 
            (words["second"] == second) & 
            (words["third"] == third) & 
            (words["forth"] == forth)]) > 0:
        return words[
            (words["first"] == first) & 
            (words["second"] == second) & 
            (words["third"] == third) & 
            (words["forth"] == forth)].values[0]
    else:
        return words[
            (words["first"] == first) & 
            (words["second"] == second) & 
            (words["third"] == third)].values[0]
        

### 1st guess

In [ ]:
print("First guess: " + "".join(get_first_suggestion()))

### 2nd guess

In [ ]:
suggestion = get_suggestion(
    variables=["first", "second", "third", "forth", "fifth"], 
    evidence={
    }, 
    must_contain=["S"],
    must_not_contain=["A", "B", "O", "R"], 
    must_not_contain_at={
        "first":["S"], 
        "second":[], 
        "third":[],
        "forth":[],
        "fifth":[]
    }
)

print("Next guess: " + "".join(suggestion))

### 3rd guess

In [ ]:
suggestion = get_suggestion(
    variables=["first", "second", "third", "forth", "fifth"], 
    evidence={
    }, 
    must_contain=["S", "E", "I"],
    must_not_contain=["A", "B", "O", "R", "L", "N"], 
    must_not_contain_at={
        "first":["S"], 
        "second":["E"], 
        "third":["I"],
        "forth":[],
        "fifth":["S"]
    }
)

print("Next guess: " + "".join(suggestion))

### 4th guess

In [ ]:
suggestion = get_suggestion(
    variables=["first"], 
    evidence={
        "second": "I", 
        "third": "S", 
        "forth": "T", 
        "fifth": "E"
    }, 
    must_contain=["S", "E", "I"],
    must_not_contain=["A", "B", "O", "R", "L", "N", "M"], 
    must_not_contain_at={
        "first":["S"], 
        "second":["E"], 
        "third":["I"],
        "forth":[],
        "fifth":["S"]
    }
)

print("Next guess: " + "".join(suggestion))